In [1]:
import json
import os
import pickle
from collections import defaultdict

import matplotlib
import matplotlib.pyplot as plt
import obspy
import pandas as pd
from obspy.clients.fdsn import Client


In [2]:
region_name = os.environ.get('REGION_NAME', 'Demo')
num_parallel = int(os.environ.get('NUM_PARALLEL', '1'))
bucket_name = os.environ.get('BUCKET_NAME', 'catalogs')
s3_url = os.environ.get('S3_URL', 'minio-service:9000')
secure = os.environ.get('SECURE', 'false').lower() == 'true'

In [ ]:
# Minioクライアントの設定
try:
    from minio import Minio

    minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
    if not minioClient.bucket_exists(bucket_name):
        minioClient.make_bucket(bucket_name)
        
    # config.jsonをダウンロード
    config_json = "config.json"
    minioClient.fget_object(
        "catalogs",  # バケット名
        "Demo/config.json",  # オブジェクト名
        config_json  # ローカルの保存先
    )
except Exception as err:
    print(f"ERROR: can not access minio service! \n{err}")
    pass

# 標準カタログのパス設定
standard_catalog = "catalog.csv"


In [ ]:

# matplotlib.use("agg")
# %matplotlib inline

with open(config_json, "r") as fp:
    config = json.load(fp)

####### IRIS catalog ########
try:
    events = Client(config["client"]).get_events(
        starttime=config["starttime"],
        endtime=config["endtime"],
        minlongitude=config["xlim_degree"][0],
        maxlongitude=config["xlim_degree"][1],
        minlatitude=config["ylim_degree"][0],
        maxlatitude=config["ylim_degree"][1],
        # filename='events.xml',
    )
except:
    events = Client("iris").get_events(
        starttime=config["starttime"],
        endtime=config["endtime"],
        minlongitude=config["xlim_degree"][0],
        maxlongitude=config["xlim_degree"][1],
        minlatitude=config["ylim_degree"][0],
        maxlatitude=config["ylim_degree"][1],
        # filename='events.xml',
    )

#     events = obspy.read_events('events.xml')
print(f"Number of events: {len(events)}")
#     events.plot('local', outfile="events.png")
#     events.plot('local')

####### Save catalog ########
catalog = defaultdict(list)
for event in events:
    if len(event.magnitudes) > 0:
        catalog["time"].append(event.origins[0].time.datetime)
        catalog["magnitude"].append(event.magnitudes[0].mag)
        catalog["longitude"].append(event.origins[0].longitude)
        catalog["latitude"].append(event.origins[0].latitude)
        catalog["depth(m)"].append(event.origins[0].depth)
catalog = pd.DataFrame.from_dict(catalog).sort_values(["time"])
catalog.to_csv(
    standard_catalog,
    # sep="\t",
    index=False,
    float_format="%.3f",
    date_format="%Y-%m-%dT%H:%M:%S.%f",
    columns=["time", "magnitude", "longitude", "latitude", "depth(m)"],
)

## upload to s3 bucket
try:
    from minio import Minio

    minioClient = Minio(s3_url, access_key="minio", secret_key="minio123", secure=secure)
    if not minioClient.bucket_exists(bucket_name):
        minioClient.make_bucket(bucket_name)

    minioClient.fput_object(
        bucket_name,
        f"{config['region']}/standard_catalog.csv",
        standard_catalog,
    )

except Exception as err:
    print(f"ERROR: can not access minio service! \n{err}")
    pass

####### Plot catalog ########
plt.figure()
plt.plot(catalog["longitude"], catalog["latitude"], ".", markersize=1)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.axis("scaled")
plt.xlim(config["xlim_degree"])
plt.ylim(config["ylim_degree"])
# plt.savefig(os.path.join(data_path, "events_loc.png"))
plt.show()

plt.figure()
plt.plot_date(catalog["time"], catalog["magnitude"], ".", markersize=1)
plt.gcf().autofmt_xdate()
plt.ylabel("Magnitude")
plt.title(f"Number of events: {len(events)}")
# plt.savefig(os.path.join(data_path, "events_mag_time.png"))
plt.show()

In [ ]:
# Kubeflow Pipelines UI用のメタデータ出力
if os.environ.get('ELYRA_RUNTIME_ENV') == 'kfp':
    # For information about Elyra environment variables refer to
    # https://elyra.readthedocs.io/en/stable/user_guide/best-practices-file-based-nodes.html#proprietary-environment-variables

    metadata = {
        'outputs': [
            {
                'storage': 'inline',
                'source': f'# Configuration Setup Complete\n'
                         f'- **Region**: {region_name}\n'
                         f'- **Time range**: {config["starttime"]} to {config["endtime"]}\n'
                         f'- **Parallel processes**: {num_parallel}\n'
                         f'- **Time slots**: {len(starttimes)}\n'
                         f'- **Networks**: {", ".join(network_list)}\n'
                         f'- **Center**: {center}',
                'type': 'markdown',
            }
        ]
    }

    try:
        with open('mlpipeline-ui-metadata.json', 'w') as f:
            json.dump(metadata, f)
        print("Kubeflow Pipelines UI metadata saved successfully")
    except Exception as err:
        print(f"WARNING: Could not save UI metadata: {err}")

print("Configuration setup completed successfully!")